In [1]:
import torch as th
import torch.nn as nn
import syft as sy

hook = sy.TorchHook(th)

alice = sy.VirtualWorker(hook, id="alice")
bob = sy.VirtualWorker(hook, id="bob")
charlie = sy.VirtualWorker(hook, id="charlie")
dan = sy.VirtualWorker(hook, id="dan")

In [10]:
x = th.tensor([1,2,3,4])

In [11]:
x_ptr = x.send_(bob)

In [12]:
x

(Wrapper)>[PointerTensor | me:74192600086 -> bob:45647806058]

In [5]:
x_ptr

(Wrapper)>[PointerTensor | me:28906046356 -> bob:60881848396]

In [9]:
a = sy.Promise.FloatTensor(shape=torch.Size((3, 3)))
pointer_to_a = a.send(queen)
a.keep(th.tensor([1,2,3,4,5]))

In [15]:
a = sy.Promise.FloatTensor(shape=torch.Size((3, 3)))
pointer_to_a = a.send_(queen)

a.keep(th.tensor([1,2,3,4,5])) # would return an error...
a.keep(th.tensor([1,2,3,4,5]).send(queen)) # would execute

In [6]:
def my_protocol():
    
    king = sy.VirtualWorker(hook, id="king")
    queen = sy.VirtualWorker(hook, id="queen")
    prince = sy.VirtualWorker(hook, id="prince")    
    
    # BEFORE THIS LINE HAPPENS AUTOMATICALLY
    
    # a is already a pointer
    a = syft.Promise.FloatTensor(shape=torch.Size((3, 3))).send_(queen)
    
    # b is already a pointer
    b = syft.Promise.FloatTensor(shape=torch.Size((3, 3))).send_(queen)

    x = a.move(king) # this calls .send() on queen; x is a PointerTensor to a promise
    y = b.move(king) # this calls .send() on queen; y is a PointerTensor to a promsie

    actual = x + y # this creates a new PromiseTensor on King.
                   # we get returned a PointerTensor to the promise

    z = actual.move(prince) # this calls .send() on king,

    # UNDER THE HOOD
    
    # NOTE: this needs to delete all record of .send_() arguments
    # because they were just for the construction of the protocol's inputs
    # not it's execution.
    protocol = Protocol(king, queen, prince)    
    
    for var_name, var_value in locals():
        if var_value is a pointer to a PromiseTensor
          protocol.__setattr__(var_name, var_value)
    
    return protocol

# create the protocol
protocol = my_protocol()

In [17]:
@Protocol(actors=["king", "queen", "prince"])
def my_protocol():
    
    a = syft.Promise.FloatTensor(shape=torch.Size((3, 3))).send_(queen)
    b = syft.Promise.FloatTensor(shape=torch.Size((3, 3))).send_(queen)

    x = a.move(king)
    y = b.move(king)

    actual = x + y
    
    z = actual.move(prince)

# create the protocol
protocol = my_protocol()

# assign roles to real world workers
protocol.deploy(king=charlie, queen=alice, prince=theo)

# EXECUTE:
protocol.run(a=pointer_to_input_data, 
             b=another_pointer_to_input_data,
             keep_id="run one")

# Alternative EXECUTE:

protocol.a.keep(th.tensor([1,2,3,4,5]).send(protocol.a.location), keep_id = "run one")
protocol.b.keep(th.tensor([1,2,3,4,5]).send(protocol.b.location), keep_id = "run one")

In [ ]:
a = syft.Promise.FloatTensor(shape=torch.Size((3, 3)))
pointer_to_a = a.send_obj(queen)

In [9]:
def my_method():
    x = 3
    print(locals())

In [10]:
my_method()

{'x': 3}


In [ ]:
# calling promises multiple times

a = syft.Promise.FloatTensor(shape=torch.Size((3, 3)))
b = syft.Promise.FloatTensor(shape=torch.Size((3, 3)))

y = a + b
z = y + y

a.keep(th.zeros(3,3), keep_id="first run")
b.keep(th.zeros(3,3), keep_id="second run")

b.keep(th.zeros(3,3), keep_id="first run")
a.keep(th.zeros(3,3), keep_id="second run")

In [2]:
tx = torch.tensor([[1.0, 2], [3, 4]])
ty = torch.tensor([[-8.0, -7], [6, 5]])
x.keep(tx)
y.keep(ty)

print(z.value().get())

tensor([[-7., -5.],
        [ 9.,  9.]])


In [ ]:
@syft.func2plan(args_shape=[(1,)])
def plan_alice1(in_a):
    return in_a + 1

@syft.func2plan(args_shape=[(1,)])
def plan_bob1(in_b):
    return in_b + 2

@syft.func2plan(args_shape=[(1,)])
def plan_bob2(in_b):
    return in_b + 3

@syft.func2plan(args_shape=[(1,)])
def plan_alice2(in_a):
    return in_a + 4

protocol = syft.Protocol(
    [("alice", plan_alice1), ("bob", plan_bob1), ("bob", plan_bob2), ("alice", plan_alice2)]
)
protocol.deploy(alice, bob)

x = syft.Promise.FloatTensor(shape=torch.Size((1,)))
in_ptr, res_ptr = protocol(x)

in_ptr.keep(torch.tensor([1.0]))

assert res_ptr.value().get() == 11

hook.local_worker.is_client_worker = True

In [2]:
@sy.func2plan(args_shape=[(1,)])
def inc1(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc2(x):
    return x + 1

@sy.func2plan(args_shape=[(1,)])
def inc3(x):
    return x + 1

protocol = sy.Protocol([("worker1", inc1), ("worker2", inc2), ("worker3", inc3)])

protocol.send(james)

try worker1
try worker2
try worker3


ValueError: You shouldn't use the id of a worker which is already known.
dan is in me,alice,bob,charlie,dan

In [ ]:
@syft.func2plan(args_shape=[(1,)], state=(th.tensor([1.]), ))
def plan_alice(in_a, state):
    bias, = state.read()
    return in_a + bias

@syft.func2plan(args_shape=[(1,)])
def plan_bob(in_b):
    return in_b + 2

protocol = syft.Protocol([("worker1", plan_alice), ("worker2", plan_bob)])

protocol.send(james)

In [11]:
hook.local_worker.is_client_worker = False

@syft.func2plan(args_shape=[(1,)], state=(th.tensor([1.]), ))
def plan_alice(in_a, state):
    bias, = state.read()
    return in_a + bias

@syft.func2plan(args_shape=[(1,)])
def plan_bob(in_b):
    return in_b + 2

protocol = syft.Protocol([("alice", plan_alice), ("bob", plan_bob)])

protocol.send(james)
responses = me.request_search([protocol.id], location=james)
print(responses)
ptr_protocol = responses[0]
assert isinstance(ptr_protocol, PointerProtocol)
protocol_back = ptr_protocol.get()

protocol_back.deploy(alice, bob)

print(protocol)

x = syft.Promises.FloatTensor(shape=torch.Size((1,)))
in_ptr, res_ptr = protocol_back(x)

in_ptr.keep(torch.tensor([2.0]))

assert res_ptr.value().get() == 5

hook.local_worker.is_client_worker = True

IndexError: list index out of range